In [4]:
from function_filePrep import tickerList, download_data,missing_days_andIndexTimeZone, to_month_and_add_monthYear_columns
import yfinance
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from sklearn.metrics import mean_absolute_error,mean_squared_error
import numpy as np
import matplotlib.pyplot as plt
from function_filePrep import _add_dayMonthYear 
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics

df_ticker_price = download_data(list_stocks=tickerList,start_date = '2016-01-01', end_date = '2024-09-30')
df_ticker_price1=missing_days_andIndexTimeZone(df_ticker_price)

# MAPE
def mean_absolute_percentage_error(y_true, y_pred): 
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

[*********************100%***********************]  30 of 30 completed


In [7]:
cutoffDates=pd.to_datetime(['2024-02-29','2023-12-31','2023-10-31'])
def error_rate(stockToSelect, cutoff_dates, horizon= '90 days'):
    stock = stockToSelect
    # selecting stock only - daily
    stock_hist_daily = df_ticker_price1[[stock]]
    stock_hist_daily1=_add_dayMonthYear(stock_hist_daily)
    # preparing file for prophet
    stock_hist_daily2=stock_hist_daily1.reset_index().rename(columns={'Date':'ds',stock:'y'}).copy()

    lockdowns = pd.DataFrame([
    {'holiday': 'lockdown_1', 'ds': '2020-03-21', 'lower_window': 0, 'ds_upper': '2020-06-06'},
    {'holiday': 'lockdown_2', 'ds': '2020-07-09', 'lower_window': 0, 'ds_upper': '2020-10-27'},
    {'holiday': 'lockdown_3', 'ds': '2021-02-13', 'lower_window': 0, 'ds_upper': '2021-02-17'},
    {'holiday': 'lockdown_4', 'ds': '2021-05-28', 'lower_window': 0, 'ds_upper': '2021-06-10'}
    ])
    for t_col in ['ds', 'ds_upper']:
        lockdowns[t_col] = pd.to_datetime(lockdowns[t_col])
    lockdowns['upper_window'] = (lockdowns['ds_upper'] - lockdowns['ds']).dt.days

    m = Prophet(holidays=lockdowns) 
    m.fit(stock_hist_daily2)
    cutoffs = cutoff_dates
    df_cv2 = cross_validation(m, cutoffs=cutoffs, horizon=horizon)
    df_p = performance_metrics(df_cv2)
    return df_p.loc[df_p.horizon.isin(['30 days','60 days','90 days'])]

In [15]:
cutoffDates=pd.to_datetime( ['2023-09-30','2023-03-31','2022-09-30','2022-03-31'])
error_rate('MSFT',cutoffDates)

21:06:14 - cmdstanpy - INFO - Chain [1] start processing
21:06:16 - cmdstanpy - INFO - Chain [1] done processing
  0%|          | 0/4 [00:00<?, ?it/s]21:06:16 - cmdstanpy - INFO - Chain [1] start processing
21:06:17 - cmdstanpy - INFO - Chain [1] done processing
 25%|██▌       | 1/4 [00:01<00:04,  1.49s/it]21:06:18 - cmdstanpy - INFO - Chain [1] start processing
21:06:18 - cmdstanpy - INFO - Chain [1] done processing
 50%|█████     | 2/4 [00:02<00:02,  1.26s/it]21:06:19 - cmdstanpy - INFO - Chain [1] start processing
21:06:20 - cmdstanpy - INFO - Chain [1] done processing
 75%|███████▌  | 3/4 [00:03<00:01,  1.16s/it]21:06:20 - cmdstanpy - INFO - Chain [1] start processing
21:06:21 - cmdstanpy - INFO - Chain [1] done processing
100%|██████████| 4/4 [00:04<00:00,  1.22s/it]


,horizon,mse,rmse,mae,mape,mdape,smape,coverage
20,30 days,2748.445150,52.425615,50.410366,0.183120,0.197494,0.180758,0.0
50,60 days,6128.509674,78.284798,74.652346,0.247969,0.203612,0.257342,0.0
80,90 days,7418.823826,86.132594,81.796888,0.276691,0.323714,0.282207,0.0


In [16]:
error_rate('AXP',cutoffDates)

21:09:40 - cmdstanpy - INFO - Chain [1] start processing
21:09:41 - cmdstanpy - INFO - Chain [1] done processing
  0%|          | 0/4 [00:00<?, ?it/s]21:09:41 - cmdstanpy - INFO - Chain [1] start processing
21:09:42 - cmdstanpy - INFO - Chain [1] done processing
 25%|██▌       | 1/4 [00:01<00:03,  1.18s/it]21:09:43 - cmdstanpy - INFO - Chain [1] start processing
21:09:43 - cmdstanpy - INFO - Chain [1] done processing
 50%|█████     | 2/4 [00:02<00:02,  1.20s/it]21:09:44 - cmdstanpy - INFO - Chain [1] start processing
21:09:45 - cmdstanpy - INFO - Chain [1] done processing
 75%|███████▌  | 3/4 [00:03<00:01,  1.24s/it]21:09:45 - cmdstanpy - INFO - Chain [1] start processing
21:09:46 - cmdstanpy - INFO - Chain [1] done processing
100%|██████████| 4/4 [00:04<00:00,  1.21s/it]


,horizon,mse,rmse,mae,mape,mdape,smape,coverage
20,30 days,179.639813,13.402978,12.214164,0.079509,0.075429,0.076263,0.320000
50,60 days,382.864612,19.566926,14.907109,0.095611,0.062409,0.089512,0.426667
80,90 days,1389.598590,37.277320,33.192849,0.217086,0.168249,0.203653,0.000000


In [17]:
error_rate('CAT',cutoffDates)

21:10:19 - cmdstanpy - INFO - Chain [1] start processing
21:10:20 - cmdstanpy - INFO - Chain [1] done processing
  0%|          | 0/4 [00:00<?, ?it/s]21:10:20 - cmdstanpy - INFO - Chain [1] start processing
21:10:21 - cmdstanpy - INFO - Chain [1] done processing
 25%|██▌       | 1/4 [00:01<00:04,  1.40s/it]21:10:22 - cmdstanpy - INFO - Chain [1] start processing
21:10:23 - cmdstanpy - INFO - Chain [1] done processing
 50%|█████     | 2/4 [00:02<00:02,  1.44s/it]21:10:23 - cmdstanpy - INFO - Chain [1] start processing
21:10:24 - cmdstanpy - INFO - Chain [1] done processing
 75%|███████▌  | 3/4 [00:04<00:01,  1.35s/it]21:10:24 - cmdstanpy - INFO - Chain [1] start processing
21:10:25 - cmdstanpy - INFO - Chain [1] done processing
100%|██████████| 4/4 [00:05<00:00,  1.33s/it]


,horizon,mse,rmse,mae,mape,mdape,smape,coverage
20,30 days,192.138402,13.861400,12.014576,0.056498,0.047334,0.056988,0.600000
50,60 days,811.138157,28.480487,23.732667,0.105691,0.081991,0.110427,0.133333
80,90 days,1120.179158,33.469078,30.261289,0.135221,0.102573,0.141220,0.000000


In [18]:
error_rate('WMT',cutoffDates)

21:10:58 - cmdstanpy - INFO - Chain [1] start processing
21:10:58 - cmdstanpy - INFO - Chain [1] done processing
  0%|          | 0/4 [00:00<?, ?it/s]21:10:59 - cmdstanpy - INFO - Chain [1] start processing
21:10:59 - cmdstanpy - INFO - Chain [1] done processing
 25%|██▌       | 1/4 [00:01<00:03,  1.24s/it]21:11:00 - cmdstanpy - INFO - Chain [1] start processing
21:11:01 - cmdstanpy - INFO - Chain [1] done processing
 50%|█████     | 2/4 [00:02<00:02,  1.36s/it]21:11:01 - cmdstanpy - INFO - Chain [1] start processing
21:11:02 - cmdstanpy - INFO - Chain [1] done processing
 75%|███████▌  | 3/4 [00:03<00:01,  1.28s/it]21:11:03 - cmdstanpy - INFO - Chain [1] start processing
21:11:04 - cmdstanpy - INFO - Chain [1] done processing
100%|██████████| 4/4 [00:05<00:00,  1.42s/it]


,horizon,mse,rmse,mae,mape,mdape,smape,coverage
20,30 days,12.155056,3.486410,2.930088,0.058741,0.078005,0.061290,0.48
50,60 days,17.548650,4.189111,4.023897,0.086593,0.093838,0.087382,0.12
80,90 days,24.529430,4.952719,4.211671,0.091560,0.123693,0.092911,0.25


In [19]:
error_rate('KO',cutoffDates)

21:11:23 - cmdstanpy - INFO - Chain [1] start processing
21:11:23 - cmdstanpy - INFO - Chain [1] done processing
  0%|          | 0/4 [00:00<?, ?it/s]21:11:24 - cmdstanpy - INFO - Chain [1] start processing
21:11:25 - cmdstanpy - INFO - Chain [1] done processing
 25%|██▌       | 1/4 [00:01<00:05,  1.73s/it]21:11:25 - cmdstanpy - INFO - Chain [1] start processing
21:11:26 - cmdstanpy - INFO - Chain [1] done processing
 50%|█████     | 2/4 [00:03<00:03,  1.58s/it]21:11:27 - cmdstanpy - INFO - Chain [1] start processing
21:11:28 - cmdstanpy - INFO - Chain [1] done processing
 75%|███████▌  | 3/4 [00:04<00:01,  1.49s/it]21:11:28 - cmdstanpy - INFO - Chain [1] start processing
21:11:29 - cmdstanpy - INFO - Chain [1] done processing
100%|██████████| 4/4 [00:05<00:00,  1.44s/it]


,horizon,mse,rmse,mae,mape,mdape,smape,coverage
20,30 days,15.277153,3.908600,3.357503,0.059150,0.067677,0.058458,0.24
50,60 days,7.159727,2.675767,2.581206,0.044299,0.044288,0.043712,0.24
80,90 days,3.975025,1.993746,1.855191,0.031534,0.030815,0.031045,0.54


In [20]:
error_rate('MRK',cutoffDates)

21:12:10 - cmdstanpy - INFO - Chain [1] start processing
21:12:11 - cmdstanpy - INFO - Chain [1] done processing
  0%|          | 0/4 [00:00<?, ?it/s]21:12:11 - cmdstanpy - INFO - Chain [1] start processing
21:12:12 - cmdstanpy - INFO - Chain [1] done processing
 25%|██▌       | 1/4 [00:01<00:04,  1.47s/it]21:12:13 - cmdstanpy - INFO - Chain [1] start processing
21:12:14 - cmdstanpy - INFO - Chain [1] done processing
 50%|█████     | 2/4 [00:02<00:02,  1.46s/it]21:12:14 - cmdstanpy - INFO - Chain [1] start processing
21:12:15 - cmdstanpy - INFO - Chain [1] done processing
 75%|███████▌  | 3/4 [00:03<00:01,  1.27s/it]21:12:15 - cmdstanpy - INFO - Chain [1] start processing
21:12:16 - cmdstanpy - INFO - Chain [1] done processing
100%|██████████| 4/4 [00:05<00:00,  1.30s/it]


,horizon,mse,rmse,mae,mape,mdape,smape,coverage
20,30 days,98.968546,9.948294,9.098080,0.095813,0.085026,0.095671,0.000000
50,60 days,210.488791,14.508232,12.660784,0.131236,0.153439,0.132364,0.253333
80,90 days,180.464429,13.433705,11.656950,0.115831,0.142227,0.119804,0.250000
